In [11]:
from bs4 import BeautifulSoup as bs
import pandas as pd
import requests as rq
import json
import time
import random

# Login to ResearchGate

In [1]:
from selenium import webdriver
from selenium.webdriver.firefox.service import Service as FirefoxService
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.window import WindowTypes

driver = webdriver.Firefox(service=FirefoxService(GeckoDriverManager().install())) 
driver.get("https://www.researchgate.net/login")

password = driver.find_element(By.ID, "input-password")
username = driver.find_element(By.ID, "input-login")

password.send_keys("xKCB5FpQevmMcLk")
username.send_keys("ozdi@my.hit.ac.il")
password.send_keys(Keys.RETURN)

[WDM] - Downloading: 16.9kB [00:00, 15.2MB/s]                                                                    


## Columns of the data-frame

In [12]:
fullname = []
gender = []
position = []
nation = []
institute = []
shanghai_ranking = []
depratment = []
disciplines = []
publications = []
citations = []
recommendations = []
top_cited_research = []
# top_cited_research_journal = []
interest_score = []
reads = []
h_index = []
URL = []

## Lists of institutes around the world
* *instLst_1* - Israeli universities and collages
* *instLst_2* - Ivy League universities - Eight private research universities in the Northeastern United States.
* *instLst_3* - Top internationally universities

In [23]:
instLst_1 = ["Bar Ilan University",
           "Technion-Israel Institute of Technology", "The-Open-University-of-Israel",
           "Ben-Gurion-University-of-the-Negev", "Ariel-University",
           "University-of-Haifa", "Weizmann-Institute-of-Science",
           "Academic College of Tel Aviv-Yafo", "Shamoon-College-of-Engineering"]

instLst_2 = ["Brown University", "Harvard University",
           "Yale-University", "University of Pennsylvania",
           "Columbia-University", "Geisel-School-of-Medicine",
           "Princeton University", "Cornell University"]

instLst_3 = ["Stanford-University", "Massachusetts-Institute-of-Technology",
            "University of Cambridge", "University-of-California-Berkeley",
            "University-of-Oxford", "Universite_Paris-Saclay",
            "Ecole-Polytechnique", "ETH-Zurich", "University-of-Toronto",
            "The University of Tokyo", "Tsinghua University"]
instLst_4 = ["Pontificia Universidad Católica de Chile",
                "Universidade de São Paulo",
                "Instituto Tecnológico y de Estudios Superiores de Monterrey",
                "Universidad de los Andes",
                "Universidade Estadual de Campinas",
                "Universidad Nacional Autónoma de Mexico",
                "Universidad de Chile",
                "Universidad de Buenos Aires",
                "Universidade Federal do Rio de Janeiro",
                "Universidad Nacional de Colombia",
                "UNESP",
                "Universidad de Concepción",
                "Universidad de Santiago de Chile (USACH)",
                "Universidad de Antioquia",
                "Pontificia Universidade Católica do Rio de Janeiro",
                "Pontificia Universidad Javeriana",
                "Universidade Federal de Minas Gerais",
                "Pontificia Universidad Católica del Peru",
                "Rica Universidad de Costa Rica",
                "Universidade Federal Rio Grande do Sul"]

## Shanghai ranking & Nations dict.

In [14]:
# dictionary
inst_rank = {
    "Holon-Institute-of-Technology":801,
    "Tel Aviv University":151,
    "Hebrew University of Jerusalem":77,
    "Bar Ilan University":301,
    "Technion-Israel Institute of Technology":83,
    "The-Open-University-of-Israel":201,
    "Ben-Gurion-University-of-the-Negev":401,
    "Ariel-University":801,
    "University-of-Haifa":501,
    "Weizmann-Institute-of-Science":83,
    "Academic College of Tel Aviv-Yafo":801,
    "Shamoon-College-of-Engineering":801,
    "Brown University":47,
    "Harvard University":1,
    "Yale-University":11,
    "University of Pennsylvania":15,
    "Columbia-University":8,
    "Geisel-School-of-Medicine":301,
    "Princeton University":6,
    "Cornell University":12,
    "Stanford-University":2,
    "Massachusetts-Institute-of-Technology":3,
    "University of Cambridge":4,
    "University-of-California-Berkeley":5,
    "University-of-Oxford":7,
    "Universite_Paris-Saclay":16,
    "Ecole-Polytechnique":301,
    "ETH-Zurich":20,
    "University-of-Toronto":22,
    "The University of Tokyo":24,
    "Tsinghua University":26
}
    

inst_nation = {
    "Holon-Institute-of-Technology":"Israel",
    "Tel Aviv University":"Israel",
    "Hebrew University of Jerusalem":"Israel",
    "Bar Ilan University":"Israel",
    "Technion-Israel Institute of Technology":"Israel",
    "The-Open-University-of-Israel":"Israel",
    "Ben-Gurion-University-of-the-Negev":"Israel",
    "Ariel-University":"Israel",
    "University-of-Haifa":"Israel",
    "Weizmann-Institute-of-Science":"Israel",
    "Academic College of Tel Aviv-Yafo":"Israel",
    "Shamoon-College-of-Engineering":"Israel",
    "Brown University":"United States",
    "Harvard University":"United States",
    "Yale-University":"United States",
    "University of Pennsylvania":"United States",
    "Columbia-University":"United States",
    "Geisel-School-of-Medicine":"United States",
    "Princeton University":"United States",
    "Cornell University":"United States",
    "Stanford-University":"United States",
    "Massachusetts-Institute-of-Technology":"United States",
    "University of Cambridge":"Great Britain",
    "University-of-California-Berkeley":"United States",
    "University-of-Oxford":"Great Britain",
    "Universite_Paris-Saclay":"France",
    "Ecole-Polytechnique":"France",
    "ETH-Zurich":"Switzerland",
    "University-of-Toronto":"Canada",
    "The University of Tokyo":"Japan",
    "Tsinghua University":"Chaina"
}

## Determine the gender by name using API function

In [15]:
def genderFromName(string):
    name = string.split()
    name = name[0]
#     print(name) # taking only firstname
    url = "https://api.genderize.io?name="+name
    headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64; rv:102.0) Gecko/20100101 Firefox/102.0'}
    file = rq.get(url, headers=headers)
    if str(file.content).find("female") == -1: #TODO: maybe a better way in time consuming.
        return "male"
    else:
        return "female"

## Crawling and gathering information from ResearchGate function

In [16]:
def crawler(instLst, NumOfSientistPerInstatue):
    for x in instLst:
        driver.get("https://www.researchgate.net/institution/"+x+"/members")
        content = driver.find_element(By.XPATH, '/html/body/div[1]/div[3]/div[1]/div/div/div[2]/div/div[1]/div/div[3]/div/div/nav/div[7]/a/span')
        index = bs(content.get_property("innerHTML"), "html.parser").get_text()
    #     print(index) # Number of member pages for each institue

        limit = min(int(index) + 1, NumOfSientistPerInstatue + 1) # we don't mind more than 100*10 researchers per institue

        for i in range(1, limit): #int(index) + 1

            driver.get("https://www.researchgate.net/institution/"+x+"/members/"+str(i))
            p = driver.current_window_handle

            for j in range(1, 11):

                try:
                    name = driver.find_element(By.XPATH, "/html/body/div[1]/div[3]/div[1]/div/div/div[2]/div/div[1]/div/div[2]/div/div["+str(j)+"]/div/div/div[2]/div/div[1]/div/div/h5/a")
                    url = name.get_attribute('href')
                    name = bs(name.get_property("innerHTML"), "html.parser").get_text()
                    print(url)
                    print(name)
                except:
                    break #no more people to show

                try:
                    Gender = genderFromName(name)
                except:
                    Gender = "Nan"

                print (Gender)
                print(x)
                print(inst_rank[x])
                try:
                    Department = driver.find_element(By.XPATH, "/html/body/div[1]/div[3]/div[1]/div/div/div[2]/div/div[1]/div/div[2]/div/div["+str(j)+"]/div/div/div[2]/div/div[2]/div/div[2]/ul/li/span")
                    Department = bs(Department.get_property("innerHTML"), "html.parser").get_text()
                    print(Department)
                except:
                    Department = "Nan"
                    print("Nan")

                try: 
                    Disciplines = driver.find_element(By.XPATH, "/html/body/div[1]/div[3]/div[1]/div/div/div[2]/div/div[1]/div/div[2]/div/div["+str(j)+"]/div/div/div[2]/div/div[3]/div/div[2]")
                    Disciplines = bs(Disciplines.get_property("innerHTML"), "html.parser").get_text("|", strip=True)
    #                 A = [text for text in disciplines.stripped_strings] # getting list of disciplines instead of on string
                    print(Disciplines)
                except:
                    Disciplines = "Nan"
                    print("Nan")

                # all data on the members' page was collected    
                # now we want to reach the web page of the sientist

                driver.switch_to.new_window(WindowTypes.TAB)
                driver.get(url+'/stats')

                try: 
                    Research_Interest_Score = driver.find_element(By.XPATH, "/html/body/div[1]/div[3]/div[1]/div/div/div/div[2]/div[1]/div/div/div[1]/div/div/div/div[2]/ul/li[1]/div/div/div/div[1]/div")
                    Research_Interest_Score = bs(Research_Interest_Score.get_property("innerHTML"), "html.parser").get_text()
                    if Research_Interest_Score == '- -':
                        Research_Interest_Score = "0";
                    print(Research_Interest_Score)
                except:
                    Research_Interest_Score = "0"
                    print("Nan")

                try: 
                    Reads = driver.find_element(By.XPATH, "/html/body/div[1]/div[3]/div[1]/div/div/div/div[2]/div[1]/div/div/div[1]/div/div/div/div[2]/ul/li[2]/div/div/div/div[1]/div")
                    Reads = bs(Reads.get_property("innerHTML"), "html.parser").get_text()
                    print(Reads)
                except:
                    Reads = "0"
                    print("Nan")

                try: 
                    Citations = driver.find_element(By.XPATH, "/html/body/div[1]/div[3]/div[1]/div/div/div/div[2]/div[1]/div/div/div[1]/div/div/div/div[2]/ul/li[3]/div/div/div/div[1]/div")
                    Citations = bs(Citations.get_property("innerHTML"), "html.parser").get_text()
                    print(Citations)
                except:
                    Citations = "0"
                    print("Nan")

                try: 
                    Recommendations = driver.find_element(By.XPATH, "/html/body/div[1]/div[3]/div[1]/div/div/div/div[2]/div[1]/div/div/div[1]/div/div/div/div[2]/ul/li[4]/div/div/div/div[1]/div")
                    Recommendations = bs(Recommendations.get_property("innerHTML"), "html.parser").get_text()
                    print(Recommendations)
                except:
                    Recommendations = "0"
                    print("Nan")

                try: 
                    hindex = driver.find_element(By.XPATH, "/html/body/div[1]/div[3]/div[1]/div/div/div/div[2]/div[1]/div/div/div[3]/div/div/div/div[2]/ul/li[1]/ul/li[1]/div/div/div/div[1]/div")
                    hindex = bs(hindex.get_property("innerHTML"), "html.parser").get_text()
                    print(hindex)
                except:
                    hindex = "0"
                    print("Nan")

                try: 
                    Top_cited_research = driver.find_element(By.XPATH, "/html/body/div[1]/div[3]/div[1]/div/div/div/div[2]/div[1]/div/div/div[3]/div/div/div/div[2]/ul/li[2]/div/div/div/div[2]/div/a/strong")
                    Top_cited_research = bs(Top_cited_research.get_property("innerHTML"), "html.parser").get_text()
                    print(Top_cited_research)
                except:
                    Top_cited_research = "Nan"
                    print("Nan")

                try: 
                    Position = driver.find_element(By.XPATH, "/html/body/div[1]/div[3]/div[1]/div/div/div/div[1]/div[1]/div[1]/div/div/div[2]/div/div[3]/ul/li[1]")
                    Position = bs(Position.get_property("innerHTML"), "html.parser").get_text()
                    word = Position.split()[0]
                    if word == x.split()[0] or word == x.split('-')[0]:
                        Position = "Nan"
                    print(Position)
                except:
                    Position = "Nan"
                    print("Nan")

                try: 
                    no_publications = driver.find_element(By.XPATH, '/html/body/div[1]/div[3]/div[1]/div/div/div/div[1]/div[2]/div/div/div/div[1]/div/nav/div/div[1]/a[2]/div/div')
                    no_publications = bs(no_publications.get_property("innerHTML"), "html.parser").get_text()

                    # reciving the number of publications

                    no_publications = no_publications.split()[1].strip('()')
                    print(no_publications)

                except:
                    no_publications = "0"
                    print("0")


                # now we've done scraping all relevent data on this sientinst
                # we close the tab and return to previous tab

                time.sleep(random.uniform(1.9, 2.4)) # In order to react less suspicious
                driver.close()
                driver.switch_to.window(p)

                print("\n")

                fullname.append(name)
                gender.append(Gender)
                position.append(Position)
                nation.append(inst_nation[x])
                institute.append(x)
                shanghai_ranking.append(inst_rank[x])
                depratment.append(Department)
                disciplines.append(Disciplines)
                publications.append(float(no_publications.replace(",", "").replace(".", "")))
                citations.append(float(Citations.replace(",", "").replace(".", "")))
                recommendations.append(float(Recommendations.replace(",", "").replace(".", "")))
                top_cited_research.append(Top_cited_research)
                # top_cited_research_journal +=
                interest_score.append(float(Research_Interest_Score.replace(",", "").replace(".", "")))
                reads.append(float(Reads.replace(",", "").replace(".", "")))
                h_index.append(float(hindex.replace(",", "").replace(".", "")))
                URL.append(url)


In [24]:
crawler(instLst_4, 100)

df = pd.DataFrame({"Full name":fullname, "Gender":gender, "Position":position,
                   "Nation":nation, "Institute":institute, "Shanghai_ranking":shanghai_ranking,
                   "Depratment":depratment, "Disciplines":disciplines, "Publications":publications,
                   "Citations":citations, "Recommendations":recommendations,
                   "Top cited research":top_cited_research, "Interest score":interest_score, 
                   "Reads":reads, "h index":h_index, "URL":url})

df.to_csv('sientinst_data')

NoSuchElementException: Message: Unable to locate element: /html/body/div[1]/div[3]/div[1]/div/div/div[2]/div/div[1]/div/div[3]/div/div/nav/div[7]/a/span
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:180:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:392:5
element.find/</<@chrome://remote/content/marionette/element.sys.mjs:275:16


In [20]:
df = pd.DataFrame({"Full name":fullname, "Gender":gender, "Position":position,
                   "Nation":nation, "Institute":institute, "Shanghai_ranking":shanghai_ranking,
                   "Depratment":depratment, "Disciplines":disciplines, "Publications":publications,
                   "Citations":citations, "Recommendations":recommendations,
                   "Top cited research":top_cited_research, "Interest score":interest_score, 
                   "Reads":reads, "h index":h_index, "URL":URL})

df.to_csv('sientinst_data')

In [21]:
df.to_pickle('list_1_israel.pkl')